In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

In [2]:
external_data = pd.read_csv(Path("external_data") / 'external_data.csv')
data = pd.read_parquet(Path("data") / "train.parquet")

In [3]:
external_data_cleaned = external_data.dropna(axis=1, how='all').sort_values(by='date')

In [4]:
# Définir les colonnes d'intérêt
columns_of_interest = ["date", "etat_sol", "dd", "ff", "t", "u", "vv", "n", "ht_neige", "rr3"]

# Sélectionner et copier les colonnes d'intérêt
external_data_sorted = external_data_cleaned[columns_of_interest].copy()

# Convertir la colonne 'date' en datetime
external_data_sorted["date"] = pd.to_datetime(external_data_sorted['date'])

# Convertir la température en degrés Celsius
external_data_sorted.loc[:, "t"] = external_data_sorted["t"] - 273.15

# Fonction pour définir les périodes de confinement
def set_periods(df, column_name, periods):
    df[column_name] = 0
    for start_date, end_date in periods:
        df.loc[(df['date'] >= start_date) & (df['date'] < end_date), column_name] = 1

# Définir les périodes de confinement
lockdown_periods = [
    ('2020-10-30', '2020-12-15'),
    ('2021-04-03', '2021-05-04')
]
set_periods(external_data_sorted, 'Lockdown', lockdown_periods)

# Définir les périodes de couvre-feu souple
soft_curfew_periods = [
    ('2020-10-17', '2020-10-30'),
    ('2020-12-15', '2021-01-16'),
    ('2021-05-19', '2021-06-21')
]
set_periods(external_data_sorted, 'soft-curfew', soft_curfew_periods)

# Définir les périodes de couvre-feu strict
hard_curfew_periods = [
    ('2021-01-16', '2021-04-03'),
    ('2021-05-04', '2021-05-19')
]
set_periods(external_data_sorted, 'hard-curfew', hard_curfew_periods)

# Définir les jours fériés
bank_holidays = pd.to_datetime([
    '2020-11-01', '2020-11-11', '2020-12-25', '2021-01-01', '2021-04-05',
    '2021-05-01', '2021-05-13', '2021-05-24', '2021-07-14', '2021-08-15'
])
external_data_sorted['bank_holidays'] = external_data_sorted['date'].isin(bank_holidays).astype(int)

# Set holidays dates
holidays = [
    ('2020-10-17', '2020-11-02'),
    ('2020-12-19', '2021-01-02'),
    ('2021-02-13', '2021-03-01')
    ('2021-04-17', '2021-05-03'),
    ('2021-10-23', '2021-11-08')
]
set_periods(external_data_sorted, 'holidays', holidays)

# Afficher les premières lignes du DataFrame
print(external_data_sorted.head())

<>:52: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
<>:52: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
C:\Users\thiba\AppData\Local\Temp\ipykernel_5872\393536314.py:52: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
  ('2021-02-13', '2021-03-01')
C:\Users\thiba\AppData\Local\Temp\ipykernel_5872\393536314.py:52: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
  ('2021-02-13', '2021-03-01')
C:\Users\thiba\AppData\Local\Temp\ipykernel_5872\393536314.py:52: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
  ('2021-02-13', '2021-03-01')


TypeError: 'tuple' object is not callable

from sklearn.impute import KNNImputer

numeric_columns = external_data_sorted.select_dtypes(include=[np.number]).columns
datetime_columns = external_data_sorted.select_dtypes(include=[np.datetime64]).columns

# Appliquer l'imputation par k-NN aux colonnes numériques
imputer = KNNImputer(n_neighbors=3, weights="uniform")
external_data_numeric_imputed = pd.DataFrame(imputer.fit_transform(external_data_sorted[numeric_columns]),
                                             columns=numeric_columns)

# Recombiner les colonnes numériques imputées avec les colonnes datetime
external_data_inputed = pd.concat([external_data_numeric_imputed, external_data_sorted[datetime_columns]], axis=1)

# Afficher le DataFrame imputé
print(external_data_inputed)

external_data_inputed.isna().sum()

In [ ]:
# Let's fill missing hours : we will create new lines, which will be copies of the existing lines
def create_missing_hours(row):
    new_rows = []

    # copy = existing line minus two hours
    new_row_2h = row.copy()
    new_row_2h['date'] = row['date'] - pd.Timedelta(hours=2)
    new_rows.append(new_row_2h)

    # copy = existing line minus one hour
    new_row_1h = row.copy()
    new_row_1h['date'] = row['date'] - pd.Timedelta(hours=1)
    new_rows.append(new_row_1h)

    return new_rows

# Appliquer la fonction à chaque ligne du DataFrame
new_rows = []
for index, row in external_data_sorted.iterrows():
    new_rows.extend(create_missing_hours(row))

# Convertir la liste de nouvelles lignes en DataFrame
new_data = pd.DataFrame(new_rows)

# Concaténer les nouvelles lignes avec le DataFrame original
external_data_expanded = pd.concat([external_data_sorted, new_data], ignore_index=True)

# Trier le DataFrame par date
external_data_expanded = external_data_expanded.sort_values(by='date').reset_index(drop=True)

# Afficher les premières lignes du DataFrame modifié
print(external_data_expanded.head())

                 date  etat_sol   dd   ff     t   u     vv    n  ht_neige  \
0 2020-08-31 22:00:00       0.0  340  1.6  12.6  81  30000  0.0       0.0   
1 2020-08-31 23:00:00       0.0  340  1.6  12.6  81  30000  0.0       0.0   
2 2020-09-01 00:00:00       0.0  340  1.6  12.6  81  30000  0.0       0.0   
3 2020-09-01 01:00:00       0.0  290  1.1  10.8  88  25000  0.0       0.0   
4 2020-09-01 02:00:00       0.0  290  1.1  10.8  88  25000  0.0       0.0   

   rr3  Lockdown  soft-curfew  hard-curfew  bank_holidays  holidays  
0  0.0         0            0            0              0         0  
1  0.0         0            0            0              0         0  
2  0.0         0            0            0              0         0  
3  0.0         0            0            0              0         0  
4  0.0         0            0            0              0         0  


In [ ]:
data_train = pd.merge(data, external_data_expanded, on='date', how='inner')

In [ ]:
def _encode_dates(X):
    X = X.copy()  # modify a copy of X
    X["year"] = X["date"].dt.year
    X["month"] = X["date"].dt.month
    X["day"] = X["date"].dt.day
    X["weekday"] = X["date"].dt.weekday
    X["hour"] = X["date"].dt.hour

    return X

external_data_train = _encode_dates(data_train)
external_data_train.head()

,counter_id,counter_name,site_id,site_name,bike_count,date,counter_installation_date,coordinates,counter_technical_id,latitude,...,Lockdown,soft-curfew,hard-curfew,bank_holidays,holidays,year,month,day,weekday,hour
0,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 02:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,0,0,0,0,0,2020,9,1,1,2
1,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,1.0,2020-09-01 03:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,0,0,0,0,0,2020,9,1,1,3
2,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 04:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,0,0,0,0,0,2020,9,1,1,4
3,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,4.0,2020-09-01 15:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,0,0,0,0,0,2020,9,1,1,15
4,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,9.0,2020-09-01 18:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,...,0,0,0,0,0,2020,9,1,1,18


In [ ]:
y = external_data_train["log_bike_count"]
X = external_data_train.drop(
    columns=["log_bike_count", "bike_count", "site_id", "counter_name", "site_name",
             "coordinates", "counter_technical_id", "counter_installation_date"])

In [ ]:
X = pd.get_dummies(X, columns=["counter_id"], dummy_na=True, drop_first=True, prefix_sep=' ')
X.drop(columns=["counter_id nan"], inplace=True)

In [ ]:
def train_test_split_temporal(X, y, delta_threshold="30 days"):

    cutoff_date = X["date"].max() - pd.Timedelta(delta_threshold)
    mask = (X["date"] <= cutoff_date)
    X_train, X_valid = X.loc[mask], X.loc[~mask]
    y_train, y_valid = y[mask], y[~mask]

    return X_train.drop(columns=["date"]), y_train.drop(columns=["date"]), X_valid.drop(columns=["date"]), y_valid.drop(columns=["date"])

In [ ]:
X_train, y_train, X_valid, y_valid = train_test_split_temporal(X, y)

In [ ]:
import xgboost as xgb
from sklearn.metrics import r2_score

# Initialiser le modèle XGBoost
boost = xgb.XGBRegressor(random_state=0)


# Entraîner le modèle
boost.fit(X_train, y_train)


# Faire des prédictions sur l'ensemble de test
y_pred = boost.predict(X_valid)


# Calculer le score R²
score = r2_score(y_valid, y_pred)


print("R² Score:", score)

R² Score: 0.8916057331868346


In [ ]:
import xgboost as xgb
from sklearn.metrics import r2_score

boost_v2 = xgb.XGBRegressor(learning_rate=0.13, max_depth=7, min_child_weight=3,
                            n_estimators=300, subsample=1, random_state=0)


boost_v2.fit(X_train, y_train)


y_pred2 = boost_v2.predict(X_valid)

score2 = r2_score(y_valid, y_pred2)

print("R² Score2:", score2)

R² Score2: 0.9019529474666381


import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score

# Définir le modèle XGBoost
regr = xgb.XGBRegressor(random_state=0)

# Définir la grille de paramètres
param_grid = {
    'n_estimators': [100, 300],
    'learning_rate': [0.1, 0.13],
    'max_depth': [6, 7],
    'min_child_weight': [1, 3],
    'subsample': [0.8, 1.0]
}

# Configurer GridSearchCV
grid_search = GridSearchCV(estimator=regr, param_grid=param_grid, scoring='r2', cv=3, n_jobs=-1, verbose=2)

# Entraîner le modèle avec GridSearchCV
grid_search.fit(X_train, y_train)

# Afficher les meilleurs paramètres et le meilleur score
print("Best parameters found: ", grid_search.best_params_)
print("Best R² score: ", grid_search.best_score_)

# Évaluer le modèle sur l'ensemble de test
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_valid)
test_score = r2_score(y_valid, y_pred)
print("Test R² score: ", test_score)

In [ ]:
from sklearn.metrics import root_mean_squared_error

print(f"Train set, RMSE={root_mean_squared_error(y_train, boost_v2.predict(X_train)):.4f}")
print(f"Test set, RMSE={root_mean_squared_error(y_valid, boost_v2.predict(X_valid)):.4f}")

print(f"Train set, RMSE={root_mean_squared_error(y_train, boost.predict(X_train)):.4f}")
print(f"Test set, RMSE={root_mean_squared_error(y_valid, boost.predict(X_valid)):.4f}")

Train set, RMSE=0.3729
Test set, RMSE=0.4513
Train set, RMSE=0.4453
Test set, RMSE=0.4745


from sklearn.model_selection import learning_curve

N, train_score, val_score = learning_curve(regr, X_train, y_train, train_sizes=np.linspace(0.2, 1, 5), cv=5)

plt.plot(N, train_score.mean(axis=1), label='train')
plt.plot(N, val_score.mean(axis=1), label='validation')
plt.xlabel('train_sizes')
plt.legend()